In [ ]:
# https://consensus.app/search/

# Imports installs etc

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers
!pip install --upgrade transformers
!pip install --upgrade accelerate
!pip install -q PyPDF2
!pip install -q langchain

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sklearn.metrics.pairwise import cosine_similarity

import re
import os
import ast

import json
import PyPDF2
import pandas as pd
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm
import numpy as np
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


!pip install -q PyPDF2
!pip install -q langchain

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#Embedding model utils

In [ ]:
embeddings_model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-base-en",
    trust_remote_code=True)
embeddings_model.max_seq_length = 4096

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/72.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + ' '
    text = text[:text.find("References")]
    return text

def make_embeddings_for_new_papers(papers_not_havin_embeds):
  df = pd.DataFrame()
  all_embeddings = []
  all_paragraphs = []
  all_names = []
  all_paths = []
  for i, paper in tqdm(enumerate(papers_not_havin_embeds)):
    pdf_path = paper
    extracted_text = extract_text_from_pdf(pdf_path)
    paragraphs = extracted_text.split(".\n")
    paragraphs = [tx for tx in paragraphs if len(''.join(tx.split())) > 20] # to exclude table things
    embeddings = embeddings_model.encode(paragraphs)
    embeds_row = [list(embeddings[i]) for i in range(0,len(embeddings))]
    all_embeddings.extend(embeds_row)
    all_paragraphs.extend(paragraphs)
    paths = [paper]*len(paragraphs)
    all_paths.extend(paths)
    names = ["".join(paper.split("/")[-1].split(".")[:-1])]*len(paragraphs)
    all_names.extend(names)

  df["name"] = all_names
  df["path"] = all_paths
  df["embeds"] = all_embeddings
  df["paragraphs"] = all_paragraphs
  return df

In [ ]:
def check_and_add_embeds(root_path):
  all_papers = os.listdir(root_path)
  all_paper_pdf_names = [root_path +paper for paper in all_papers if ".pdf" in paper]
  read = pd.read_csv(root_path+"embeddings.csv",lineterminator='\n')
  read["embeds"] = read["embeds"].apply(ast.literal_eval)
  papers_not_havin_embeds = [paper for paper in all_paper_pdf_names if paper not in read["path"].tolist()]
  if len(papers_not_havin_embeds):
    print("NEW papers: ", papers_not_havin_embeds)
    additional_table = make_embeddings_for_new_papers(papers_not_havin_embeds)
    df = pd.concat([read, additional_table])
    df.to_csv(root_path+"embeddings.csv", index=False, escapechar="\\")
    return df
  else:
    print("NO new papers")
    return read

In [ ]:
def compute_cosine_similarities(query_embedding, df_embeddings):
  query_embedding_reshaped = query_embedding.reshape(1, -1)
  similarities = cosine_similarity(query_embedding_reshaped, df_embeddings)[0]
  return similarities

def find_similar_papers(query, data):
  query_embeds = embeddings_model.encode(query)
  df_embeddings = np.stack(data['embeds'].values)
  similarities = compute_cosine_similarities(query_embeds, df_embeddings)
  data_new = data.copy()
  data_new['similarity'] = similarities
  data_new = data_new.sort_values(by='similarity', ascending=False)

  top_5_similar = data_new.head(5)
  context = ""
  source_papers = ""
  for i in range(len(top_5_similar)):
    context += top_5_similar.iloc[i]["paragraphs"] + ". "
    source_papers += top_5_similar.iloc[i]["name"] + ", "
    print(top_5_similar.iloc[i]["similarity"])
  return context, source_papers

# Here goes the funny part

In [ ]:
root_path = "/content/drive/MyDrive/papers_for_workshop/"
data = check_and_add_embeds(root_path)

NO new papers


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=5000,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)
llm = HuggingFacePipeline(pipeline=pipeline)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
#### Prompt
template = """<s>[INST]
You are a helpful, respectful and honest assistant which knows the data based on the following articles.
Answer exactly in few words from the context
Answer the question below from context below :
{context}
{question} [/INST] </s>
"""

In [ ]:
example_first = "What is the MultiMedQA benchmark, and how does it enhance the evaluation of large language models (LLMs) in medical and clinical applications, specifically in the context of the performance of PaLM and its variants?"

In [ ]:
for i in range(10):
  print("Ask a question")
  question_p = input()
  if question_p == "exit":
    break
  # print("Context: ",
  context_p, source_papers = find_similar_papers (question_p, data)
  context_p = ""
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question_p,"context":context_p})
  print(response)
  print("source papers: ", source_papers)

Ask a question
What is the MultiMedQA benchmark, and how does it enhance the evaluation of large language models (LLMs) in medical and clinical applications, specifically in the context of the performance of PaLM and its variants?
0.8869489429281731
0.8598360668295164
0.8460381054742474
0.8451182902831059
0.8433880696542518


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



The MultiMedQA benchmark evaluates large language models (LLMs) for question-answering tasks in the medical and clinical domain. It enhances the evaluation by providing a comprehensive and diverse set of tasks, and by comparing LLMs to human expert answers. Specifically, it compares the performance of LLMs like PaLM and its variants.
source papers:  Large Language Models Encode Clinical Knowledge , Large Language Models Encode Clinical Knowledge , Large Language Models Encode Clinical Knowledge , PaLM 2 Technical Report, Large Language Models Encode Clinical Knowledge , 
Ask a question
exit
